In [1]:
import numpy as np

In [2]:
# граф задан вектором смежности
graph = {'A': ['C', 'M'], 'B': ['M'], 'C':[], 'D':['C'], 'E':['C', 'I', 'J'], 'F':['D', 'K', 'L'], 
    'G': ['K'], 'H':['B', 'D', 'F', 'G'], 'I':['A', 'N'], 'J':['A', 'G', 'L'], 'K':['C'],
    'L':[], 'M':['C'], 'N':['F']}

In [3]:
# алгоритм Демукрона записан для матрицы смежности.
# Конвертируем вектор смежности в матрицу смежности
def convert_to_matrix(graph):
    map_g = {}
    for ind, item in enumerate(graph.keys()):
        map_g[item] = ind
    arr = np.zeros([len(graph), len(graph)],int)
    for ind, item in graph.items():
        for element in item:
            arr[map_g[ind]][map_g[element]] = 1
    return arr

In [4]:
graph_ms = convert_to_matrix(graph)

In [5]:
graph_ms

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

## Алгоритм Демукрона

In [6]:
def demukron(graph):
    exit_half_dg =  graph.sum(axis=0)
    result = list()
    vertex = set()
    while exit_half_dg.sum() > 0:
        layer, = np.where(exit_half_dg == 0)
        temp = list(set(layer) - vertex)
        result.append(temp)
        vertex.update(layer)
        exit_half_dg = exit_half_dg - graph[temp].sum(axis=0)
    layer, = np.where(exit_half_dg == 0)
    temp = list(set(layer) - vertex)
    result.append(temp)
    return result 

In [7]:
answer = demukron(graph_ms); answer

[[4, 7], [8, 1, 9], [0, 13, 6], [12, 5], [11, 10, 3], [2]]

In [8]:
f = {'0':'A', '1':'B', '2':'C', '3':'D', '4':'E', '5':'F', '6':'G', 
     '7':'H', '8':'I', '9':'J', '10':'K', '11':'L', '12':'M', '13':'N'}

In [9]:
result = {}
for ind, row in enumerate(answer):
    result[f'layer_{ind}'] = [f[str(item)] for item in row]
result

{'layer_0': ['E', 'H'],
 'layer_1': ['I', 'B', 'J'],
 'layer_2': ['A', 'N', 'G'],
 'layer_3': ['M', 'F'],
 'layer_4': ['L', 'K', 'D'],
 'layer_5': ['C']}

## Алгоритм Тарьяна

In [10]:
from random import randint

In [11]:
class Node:
    def __init__(self, key):
        self.key = key
        self.edges = None
        self.next = None
        self.color = 'white'
        
    def __repr__(self):
        return f'Node "{self.key}"'
    
    def __iter__(self):
        item = self.edges
        while item is not None:
            yield item
            item = item.next
    
        
class Edge:
    def __init__(self, node_in, weight=None, key=None):
        self.next = None
        self.node_in = node_in
        self.weight = weight
        self.key = key
        
    def __repr__(self):
        return f'Edge "key-{self.key}, weight-{self.weight}, to-{self.node_in}"'
        
class Graph:
    def __init__(self, node=None, adges=Node):
        self.head_node = node
        
    def __iter__(self):
        item = self.head_node
        while item is not None:
            yield item
            item = item.next
            
    def add_node(self, node):
        if self.head_node is None:
            self.head_node = node
            return
        last_node = self.head_node
        while last_node.next:
            last_node = last_node.next
        last_node.next = node
    
    
    def search_node(self, key):
        if self.head_node is None:
            return
        next_node = self.head_node
        if next_node.key == key:
            return next_node
        while next_node.next:
            next_node = next_node.next
            if next_node.key == key:
                return next_node
    
    
    def add_edge(self, node_key_out, node_key_in,  weight=None, key=None):
        node_out = self.search_node(node_key_out)
        node_in = self.search_node(node_key_in)
        edge = Edge(node_in, weight, key)
        if not node_out or not node_in:
            print('Не найдена вершина')
            return
        if node_out.edges is None:
            node_out.edges = edge
            return
        next_edge = node_out.edges
        while next_edge.next:
            next_edge = next_edge.next
        next_edge.next = edge
        
    def remove_edge_out(self, node_key_out, node_key_in):
        node_out = self.search_node(node_key_out)
        if not node_out.edges:
            return
        next_edge = node_out.edges
        if next_edge.node_in.key == node_key_in:
            node_out.edges = next_edge.next
            return
        while next_edge.next:
            if next_edge.next.node_in.key == node_key_in:
                next_edge.next = next_edge.next.next
                return
            next_edge = next_edge.next
            
    def remove_node(self, node_key):
        self.remove_edges_in(node_key)
        
        next_node = self.head_node
        if next_node.key == node_key:
            self.head_node = next_node.next
            return
        while next_node:
            if next_node.next.key == node_key:
                next_node.next = next_node.next.next
            next_node = next_node.next
        
        
    def del_edges(self, node, node_key_in):
        if not node.edges:
            return
        edge = node.edges
        if edge.node_in.key == node_key_in:
            node.edges = edge.next
        while edge.next:
            if edge.next.node_in.key == node_key_in:
                edge.next = edge.next.next
                continue
            edge = edge.next
    
    def remove_edges_in(self, node_key_in):
        if not self.head_node:
            return       
        node = self.head_node
        while node:
            self.del_edges(node, node_key_in)            
            node = node.next
            
    def get_adjacent_nodes(self, node):
        return [item.node_in for item in node]
    
    def refresh_color(self):
        for item in self:
            item.color = 'white'
            
    def nodes_list(self):
        return [item.key for item in self]


In [12]:
class Nod:
    def __init__(self, item, nextt=None):
        self._item = item
        self._nextt = nextt
        
    def get_item(self):
        return self._item
    
    def get_next(self):
        return self._nextt
    
    def set_next(self, nextt):
        self._nextt = nextt
        

class Stack:
    def __init__(self, head=None):
        self._head = head
        
    
    def push(self, item):
        self._head = Nod(item, self._head)
        
    def pop(self):
        if (self.is_empty()):
            return None
        item = self._head.get_item()
        self._head = self._head.get_next()
        return item
        
    def is_empty(self):
        return self._head == None
    

class Queue:
    def __init__(self, head=None, tail=None):
        self._head = head
        self._tail = tail
        
    def is_empty(self):
        return self._head  == None
    
    def get_head(self):
        return self._head
    
    def enqueue(self, item):
        node = Nod(item)
        if self.is_empty():
            self._head = self._tail = node
        else:
            self._tail.set_next(node)
            self._tail = node
            
    def dequeue(self):
        if self.is_empty():
            return None
        item = self._head.get_item()
        self._head = self._head.get_next()
        return item
    
    def contain(self, item):
        node = self.get_head()
        while node:
            if node.get_item() == item:
                return True
            node = node.get_next()
        return False

In [13]:
def zero_half_degree(graph):
    entry = {}
    for item in graph:
        entry[item.key] = 0
    for item in graph:
        for edge in item:
            entry[edge.node_in.key] += 1
    for key, value in entry.items():
        if value == 0:
            return key

In [14]:
def DFS_s(graph, node, stack, visited):
    if node.color != 'white':
        return
    node.color = 'gray'
    adjacent_nodes = graph.get_adjacent_nodes(node)
    for item in adjacent_nodes:
        if item.color == 'white':
            DFS_s(graph, item, stack, visited)
    node.color = 'black'
    visited.append(node.key)
    stack.push(node.key)

In [15]:
def check_zero_dg(node_key, node_list, graph):
    for item in node_list:
        vertex = graph.search_node(item)
        for edge in vertex:
            if edge.node_in.key == node_key:
                return False
    return True       

In [16]:
def tarian(graph):
    zero_entry = zero_half_degree(graph)
    if not zero_entry:
        print('Graph has no vertex with a zero semi-degrees of entry.')
        return
    node = graph.search_node(zero_entry)
    
    visited = []
    result = []
    stack = Stack()
    nodes_key_list = graph.nodes_list()
    list_for_search = nodes_key_list.copy()
    len_nodes_key_list = len(nodes_key_list)
    while len(visited) < len_nodes_key_list:
        
        DFS_s(graph, node, stack, visited)
        list_for_search = list(set(nodes_key_list) - set(visited))
        for item_key in list_for_search:
            if check_zero_dg(item_key, list_for_search, graph):
                node = graph.search_node(item_key)
                if node:
                    break
    while not stack.is_empty():
        result.append(stack.pop())        
    graph.refresh_color()
    return result

In [17]:
g = Graph()

g.add_node(Node('A'))
g.add_node(Node('B'))
g.add_node(Node('C'))
g.add_node(Node('D'))
g.add_node(Node('E'))
g.add_node(Node('F'))
g.add_node(Node('G'))
g.add_node(Node('H'))

g.add_edge('E','A')
g.add_edge('A','B')
g.add_edge('B','C')
g.add_edge('D','C')
g.add_edge('C','D')
g.add_edge('H','D')
g.add_edge('B','E')
g.add_edge('E','F')
g.add_edge('B','F')
g.add_edge('G','F')
g.add_edge('F','G')
g.add_edge('C','G')
g.add_edge('H','G')
g.add_edge('D','H')

In [18]:
tarian(g)

Graph has no vertex with a zero semi-degrees of entry.


In [19]:
g2 = Graph()

g2.add_node(Node('A'))
g2.add_node(Node('B'))
g2.add_node(Node('C'))
g2.add_node(Node('D'))
g2.add_node(Node('E'))
g2.add_node(Node('F'))
g2.add_node(Node('G'))
g2.add_node(Node('H'))

g2.add_edge('A','E')
g2.add_edge('A','F')
g2.add_edge('A','G')
g2.add_edge('B','A')
g2.add_edge('B','D')
g2.add_edge('B','F')
g2.add_edge('C','G')
g2.add_edge('C','H')
g2.add_edge('H','G')
g2.add_edge('H','D')


In [20]:
tarian(g2)

['C', 'H', 'B', 'D', 'A', 'G', 'F', 'E']

In [21]:
g3 = Graph()

g3.add_node(Node('A'))
g3.add_node(Node('B'))
g3.add_node(Node('C'))
g3.add_node(Node('D'))
g3.add_node(Node('E'))
g3.add_node(Node('F'))
g3.add_node(Node('G'))
g3.add_node(Node('H'))

g3.add_edge('A','E')
g3.add_edge('A','F')
g3.add_edge('A','G')
g3.add_edge('B','A')
g3.add_edge('B','D')
g3.add_edge('B','F')
g3.add_edge('C','G')
g3.add_edge('C','H')
g3.add_edge('C','B')
g3.add_edge('F','E')
g3.add_edge('F','G')
g3.add_edge('H','G')
g3.add_edge('H','D')

In [22]:
tarian(g3)

['C', 'B', 'A', 'F', 'E', 'H', 'D', 'G']